In [1]:
import logging
import os
import sys
from io import BytesIO
import pandas as pd
import requests
from datetime import date, timedelta, datetime
from collections import Counter
import numpy as np
from dotenv import load_dotenv
from tns_api import get_object
import json
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.coordinates import SkyCoord
from tqdm import tqdm
import time

In [2]:
load_dotenv()

True

In [3]:
TNS_API_KEY= os.getenv("api_key")  # Works on real and sandbox now
TNS_BOT_NAME = os.getenv("name")
TNS_BOT_ID = os.getenv("tns_id")

In [4]:
CATALOG_URL = 'https://www.wis-tns.org/system/files/tns_public_objects/tns_public_objects.csv.zip'

In [5]:
def download_table():
    with requests.post(
        CATALOG_URL,
        headers={
            'user-agent': 'tns_marker{{"tns_id":"{id}","type": "bot", "name":"{name}"}}'.format(
                id=TNS_BOT_ID,
                name=TNS_BOT_NAME,
            )
        },
        data={
            'api_key': (None, TNS_API_KEY)
        },
    ) as response:
        return response.content

In [6]:
data = download_table()

In [7]:
df = pd.read_csv(
        BytesIO(data),
        skiprows=1,
        compression='zip',
    )

In [8]:
date.today()

datetime.date(2024, 12, 5)

In [9]:
df["creationdate"] = pd.to_datetime(df["creationdate"])
df["lastmodified"] = pd.to_datetime(df["lastmodified"])

In [10]:
now = datetime.now()
last_week = now - timedelta(days=7)

In [11]:
last_week_rows = df[df['lastmodified'] > last_week]

In [12]:
last_week_rows.columns

Index(['objid', 'name_prefix', 'name', 'ra', 'declination', 'redshift',
       'typeid', 'type', 'reporting_groupid', 'reporting_group',
       'source_groupid', 'source_group', 'discoverydate', 'discoverymag',
       'discmagfilter', 'filter', 'reporters', 'time_received',
       'internal_names', 'Discovery_ADS_bibcode', 'Class_ADS_bibcodes',
       'creationdate', 'lastmodified'],
      dtype='object')

In [13]:
last_week_visible = last_week_rows.iloc[np.where(last_week_rows["declination"] > -30)]

In [14]:
def is_close(distance, brightness):
    print(distance)
    print(brightness)
    print((10 / (brightness - 3)))
    print(np.log10(distance))
    return ((10 / (brightness - 3)) > np.log10(distance))

In [15]:
def get_bright_star(sky_object, cone = 5*u.arcsec):
    result = Vizier.query_region(sky_object,
                             radius=cone,
                             catalog='I/355/gaiadr3',
                             column_filters={'Gmag': '<15'})
    if len(result) == 0:
        return False
    return len(result[0]) > 0

In [17]:
too_close = []
for i in tqdm(SkyCoord(last_week_visible["ra"], last_week_visible["declination"], frame = "icrs", equinox = 'J2000', unit="deg")):
    too_close.append(get_bright_star(i))
    time.sleep(2.1)

100%|██████████| 799/799 [33:37<00:00,  2.53s/it]


In [18]:
too_close = np.asarray(too_close)

In [19]:
last_week_visible_seperate = last_week_visible.iloc[np.where(too_close == False)]

In [20]:
Counter(last_week_visible_seperate["reporting_group"])

Counter({'XOSS': 18,
         'GOTO': 24,
         'YSE': 43,
         'GaiaAlerts': 23,
         'BlackGEM': 6,
         'ALeRCE': 110,
         'ZTF': 211,
         'ATLAS': 99,
         'Pan-STARRS': 251,
         'MASTER': 3,
         'SGLF': 6,
         nan: 1,
         'J-PAS': 1})

In [21]:
last_week_visible_seperate.iloc[np.where(last_week_visible_seperate["reporting_group"] == "ZTF")]["internal_names"]

20                                        ZTF24abvruwg
21                                        ZTF24abttpxy
24                                        ZTF24abuulvw
25                                        ZTF24absvnza
26                                        ZTF24abvbjor
                            ...                       
871       ZTF24abtddls, ATLAS24qji, GOTO24ikr, PS24loe
886                              ZTF24abhwymz, PS24lrm
891                              ZTF24abfbska, PS24lpo
894    ZTF24abeqnyi, BGEM J025642.69+061404.8, PS24lrt
903                              ZTF24abalnbv, PS24lpr
Name: internal_names, Length: 211, dtype: object

In [22]:
last_week_visible_seperate

,objid,name_prefix,name,ra,declination,redshift,typeid,type,reporting_groupid,reporting_group,...,discoverymag,discmagfilter,filter,reporters,time_received,internal_names,Discovery_ADS_bibcode,Class_ADS_bibcodes,creationdate,lastmodified
0,167852,AT,2024adhj,91.564708,50.491417,NaN,NaN,NaN,10.0,XOSS,...,18.2300,1.0,Clear,"Jianlin Xu, Mi Zhang, Guoyou Sun, Jiangao Ruan...",2024-12-03 23:56:29,KATS24X001,NaN,NaN,2024-12-03 23:56:30,2024-12-03 23:56:30
1,167851,AT,2024adhi,165.550285,-4.500053,NaN,NaN,NaN,59.0,GOTO,...,19.5600,120.0,L,"D. O'Neill, G. Ramsay, K. Ackley, M. Dyer, F. ...",2024-12-03 21:23:25,GOTO24irx,NaN,NaN,2024-12-03 21:23:27,2024-12-03 21:23:27
2,167850,AT,2024adhh,203.986204,12.525781,NaN,NaN,NaN,59.0,GOTO,...,18.7100,120.0,L,"D. O'Neill, G. Ramsay, K. Ackley, M. Dyer, F. ...",2024-12-03 21:21:07,GOTO24iqv,NaN,NaN,2024-12-03 21:21:08,2024-12-03 21:21:08
3,167849,AT,2024adhg,341.908952,17.351157,NaN,NaN,NaN,83.0,YSE,...,19.9700,56.0,g,"D. O. Jones (UC Santa Cruz), K. D. French (Car...",2024-12-03 19:48:54,PS24mgt,NaN,NaN,2024-12-03 19:48:55,2024-12-03 19:48:55
4,167848,AT,2024adhf,172.538846,-13.748777,NaN,NaN,NaN,59.0,GOTO,...,14.9000,120.0,L,"D. O'Neill, K. Ackley, M. Dyer, F. Jiménez-Iba...",2024-12-03 15:11:28,GOTO24iqy,NaN,NaN,2024-12-03 15:11:29,2024-12-03 15:11:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,159248,SN,2024rqt,239.018631,28.900501,0.061,3.0,SN Ia,74.0,ALeRCE,...,20.0561,110.0,g,"A. Munoz-Arancibia, F.E. Bauer, F. Forster, G....",2024-08-09 00:49:55,"ZTF24aazqcuz, ATLAS24mxs, PS24hxa",2024TNSTR2831....1M,"2024TNSCR3146....1T, 2024TNSCR3155....1R",2024-08-09 00:50:01,2024-11-27 23:07:37
910,158680,AT,2024qwf,19.268870,-7.906069,NaN,NaN,NaN,59.0,GOTO,...,19.0700,120.0,L,"A. Kumar, G. Ramsay, K. Ackley, M. Dyer, F. Ji...",2024-08-02 06:21:24,"GOTO24egf, ATLAS24mjy, ZTF24abcwcat, PS24lrd",2024TNSTR2713....1K,NaN,2024-08-02 06:21:25,2024-11-27 23:26:28
911,158460,SN,2024qnw,42.211565,-3.655257,0.040,107.0,SN Ib-pec,59.0,GOTO,...,18.9900,120.0,L,"A. Kumar, D. O'Neill, K. Ackley, M. Dyer, F. J...",2024-07-30 06:22:14,"GOTO24dxi, ATLAS24mkd, ZTF24aaxnosx, PS24htw",2024TNSTR2657....1K,2024TNSCR2904....1S,2024-07-30 06:22:15,2024-11-27 23:08:11
912,158051,SN,2024qar,67.949670,1.675136,0.040,3.0,SN Ia,18.0,ATLAS,...,17.9490,72.0,orange,"J. Tonry, L. Denneau, H. Weiland, A. Lawrence,...",2024-07-25 08:35:29,"ATLAS24lqh, GOTO24edk, PS24lox",2024TNSTR2574....1T,2024TNSCR2730....1T,2024-07-25 08:35:30,2024-11-27 23:10:00


In [26]:
now.date().year

2024

In [29]:
last_week_visible_seperate.to_csv("TNS_ToO/ToO_Objects-{}{:02d}{:02d}.csv".format(now.date().year, now.date().month, now.date().day), index = False)